In [ ]:
from IDCardProcessor import IDCardProcessor

processor = IDCardProcessor('IDback.jpg')
processor.process_image()
processor = IDCardProcessor('1.jpeg')
processor.process_image()
processor = IDCardProcessor('2.jpg')
processor.process_image()
processor = IDCardProcessor('3.jpg')
processor.process_image()

In [ ]:
from paddleocr import PaddleOCR, draw_ocr
from ImageProcessor import ImageProcessor
from PIL import Image

def fetch_left_text(line, res):
    right_top = [line[0][1][0]+5, line[0][1][1]-20]
    right_bottom = [line[0][2][0]+5, line[0][2][1]+50]
    address = []
    for temp in res:
        if temp[0][0][0] > right_top[0] and temp[0][0][1] > right_top[1] and temp[0][0][1] < right_bottom[1]:
            address.append(temp)
    return address

def list_to_str(list):
    str = ''
    for item in list:
        str += item[1][0]
    return str

def sim_to_tra(text):
    from opencc import OpenCC
    cc = OpenCC('s2tw')
    return cc.convert(text)

# 設置字體路徑（例如 Windows 下的 SimSun 字體）
font_path = "C:\Windows\Fonts\simsun.ttc"

# 初始化 OCR 模型
ocr = PaddleOCR(use_angle_cls=True, lang="chinese_cht")  # 加載模型
# ocr = PaddleOCR(use_angle_cls=True, lang="ch")  # 加載模型
# 載入圖片並進行 OCR
img_path = 'IDback.jpg'
ImageProcessor(img_path).correct_image_orientation()
img_path = 'corrected_orientation_image.jpg'
result = ocr.ocr(img_path, cls=True)


# 打印 OCR 結果
for idx in range(len(result)):
    res = result[idx]
    for line in res:
        print(line)
        if line[1][0] == '住址':
            address = fetch_left_text(line, res)

str_address = list_to_str(address)
str_address = sim_to_tra(str_address)
print(str_address)
# 顯示結果並繪製文字框
result = result[0]
image = Image.open(img_path).convert('RGB')
boxes = [line[0] for line in result]
txts = [line[1][0] for line in result]
scores = [line[1][1] for line in result]
im_show = draw_ocr(image, boxes, txts, scores, font_path=font_path)
im_show = Image.fromarray(im_show)
im_show.save('result.jpg')